In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import dotenv_values

c_pg = dotenv_values("cred-pg.txt")

In [ ]:
engine = create_engine(
    f'postgresql+psycopg://{c_pg["user"]}:{c_pg["password"]}@{c_pg["host"]}/{c_pg["dbname"]}', 
    connect_args = {
        'options': '-c search_path=${user},ugeobln,ugm,uinsta,umisc,umobility,usozmed,public', 
        'keepalives_idle': 120
    },
    pool_size=1, 
    max_overflow=0,
    execution_options={ 'isolation_level': 'AUTOCOMMIT' }
)

In [ ]:
def bsp_er_diagram():
    return """
        @startuml
        ' hide the spot
        hide circle

        ' avoid problems with angled crows feet
        skinparam linetype ortho

        entity "E01" as e01 {
          *e1_id : number
          --
          *name : text
          description : text
        }

        entity "E02" as e02 {
          *e2_id : number
          --
          *e1_id : number (fk)
          other_details : text
        }

        e01 ||..o{ e02
        @enduml
    """

print(bsp_er_diagram())

In [ ]:
def emit_start():
    return """
        @startuml
        ' hide the spot
        hide circle

        ' avoid problems with angled crows feet
        skinparam linetype ortho
    """   
     
def emit_end():
    return """
        @enduml
    """   
     
def emit_entity(entity):
    return f'''
        entity "{entity}" as {entity} {{
          id 
          --
          attributes
        }}
    '''   
     
def emit_entities(entities):
    return "".join([emit_entity(entity) for entity in entities])
     
def er_diagram(schema):
    with engine.connect() as con:
        sql = f"""
        select tablename 
        from pg_tables 
        where schemaname = '{schema}'
        order by tablename
        """
        df = pd.read_sql_query(text(sql), con)       
        return f"{emit_start()} {emit_entities(df.tablename.values)} {emit_end()}"

In [ ]:
print(er_diagram("umobility"))